In [1]:
import os
import joblib
import tensorflow as tf
import pandas as pd

from src.features.data_engineering import preprocess_prediction_data
from src.squad_selection.select_team import select_team, print_selected_team

In [2]:
rolling_columns = ['assists', 'bonus', 'bps', 'clean_sheets',
                   'creativity', 'goals_conceded', 'goals_scored',
                   'ict_index', 'influence', 'minutes',
                   'saves', 'selected', 'player_team_score', 'opponent_team_score', 'threat',
                   'total_points', 'transfers_in', 'transfers_out',
                   'value', 'yellow_cards']

times = ['all', 6, 3]

In [3]:
x_prediction, x_target = preprocess_prediction_data('2021-22', 30, rolling_columns=rolling_columns, rolling_times=times, opponent_team_stats=True)

Are there any NaN values? - False


In [4]:
x_target.head(10)

,name,element,team,GW,season,opponent_next_gameweek,value,position
17621,Owen Dodgson,565,Burnley,29,2021-22,Manchester City,39,DEF
17621,Owen Dodgson,565,Burnley,29,2021-22,Everton,39,DEF
17621,Owen Dodgson,565,Burnley,29,2021-22,Norwich,39,DEF
17621,Owen Dodgson,565,Burnley,29,2021-22,West Ham,39,DEF
17621,Owen Dodgson,565,Burnley,29,2021-22,Southampton,39,DEF
17621,Owen Dodgson,565,Burnley,29,2021-22,Wolverhampton Wanderers,39,DEF
17598,Sam Waller,708,Burnley,29,2021-22,Manchester City,40,GK
17598,Sam Waller,708,Burnley,29,2021-22,Everton,40,GK
17598,Sam Waller,708,Burnley,29,2021-22,Norwich,40,GK
17598,Sam Waller,708,Burnley,29,2021-22,West Ham,40,GK


In [5]:
# load 'model_mlp_rolling_opponent_stats_features' model
root_dir = os.path.dirname(os.path.abspath(''))
model = tf.keras.models.load_model(root_dir + '/models/baseline/model_mlp_rolling_opponent_stats_features.h5')
x_scaler = joblib.load(root_dir + '/models/baseline/SCALER-model_mlp_rolling_opponent_stats_features.gz')

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               12544     
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 126)               32382     
                                                                 
 dense_3 (Dense)             (None, 64)                8128      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 86,143
Trainable params: 86,143
Non-trainable params: 0
_________________________________________________________________


In [7]:
y_pred = pd.Series(model.predict(x_scaler.transform(x_prediction)).reshape(-1, ), index=x_prediction.index, name='predicted_total_points_next_gameweek')
prediction_df = pd.concat([y_pred, x_target], axis=1)

In [8]:
prediction_df

,predicted_total_points_next_gameweek,name,element,team,GW,season,opponent_next_gameweek,value,position
17621,0.000961,Owen Dodgson,565,Burnley,29,2021-22,Manchester City,39,DEF
17621,0.037667,Owen Dodgson,565,Burnley,29,2021-22,Everton,39,DEF
17621,0.051699,Owen Dodgson,565,Burnley,29,2021-22,Norwich,39,DEF
17621,0.004169,Owen Dodgson,565,Burnley,29,2021-22,West Ham,39,DEF
17621,0.013536,Owen Dodgson,565,Burnley,29,2021-22,Southampton,39,DEF
...,...,...,...,...,...,...,...,...,...
18728,0.217791,Alphonse Areola,489,West Ham,30,2021-22,Chelsea,47,GK
18674,0.044222,Jamal Baptiste,554,West Ham,30,2021-22,Everton,40,DEF
18674,0.026542,Jamal Baptiste,554,West Ham,30,2021-22,Brentford,40,DEF
18674,0.056511,Jamal Baptiste,554,West Ham,30,2021-22,Burnley,40,DEF


In [9]:
# group prediction_df by 'name', 'element', 'team', 'GW", 'season', sum up 'predicted_total_points_next_gameweek' and make 'opponent_next_gameweek' items a list
prediction_df_sum = prediction_df.groupby(['name', 'element', 'position', 'value', 'team', 'GW', 'season']).sum().reset_index()
list_of_opponents = prediction_df.groupby(['name', 'element', 'team', 'GW', 'season'])['opponent_next_gameweek'].apply(list).reset_index()['opponent_next_gameweek']

prediction_df_sum['next_4_GWs_matches'] = list_of_opponents.apply(lambda x: len(x))
prediction_df_sum['opponent_next_gameweek'] = list_of_opponents

# divide 'value' column by 100
prediction_df_sum['value'] = prediction_df_sum['value'] / 10

# replace position str by int
prediction_df_sum['position_id'] = prediction_df_sum['position'].replace({'GK': 1, 'DEF': 2, 'MID': 3, 'FWD': 4})

In [10]:
prediction_df_sum

,name,element,position,value,team,GW,season,predicted_total_points_next_gameweek,next_4_GWs_matches,opponent_next_gameweek,position_id
0,Aaron Connolly,72,FWD,5.3,Brighton,29,2021-22,0.385969,5,"[Norwich, Arsenal, Tottenham, Manchester City,...",4
1,Aaron Cresswell,411,DEF,5.4,West Ham,30,2021-22,13.169465,4,"[Everton, Brentford, Burnley, Chelsea]",2
2,Aaron Lennon,575,MID,4.9,Burnley,29,2021-22,14.788957,6,"[Manchester City, Everton, Norwich, West Ham, ...",3
3,Aaron Ramsdale,559,GK,5.1,Arsenal,30,2021-22,10.169611,5,"[Crystal Palace, Brighton, Southampton, Chelse...",1
4,Aaron Ramsey,496,MID,4.4,Aston Villa,30,2021-22,-0.048319,3,"[Wolverhampton Wanderers, Tottenham, Leicester]",3
...,...,...,...,...,...,...,...,...,...,...,...
708,Zach Awe,705,DEF,4.0,Arsenal,30,2021-22,-0.181945,5,"[Crystal Palace, Brighton, Southampton, Chelse...",2
709,Zack Steffen,260,GK,4.4,Manchester City,29,2021-22,0.340187,4,"[Burnley, Liverpool, Brighton, Watford]",1
710,Zak Swanson,711,DEF,4.0,Arsenal,30,2021-22,-0.187515,5,"[Crystal Palace, Brighton, Southampton, Chelse...",2
711,Álvaro Fernández,556,GK,4.5,Brentford,30,2021-22,0.305082,4,"[Chelsea, West Ham, Watford, Tottenham]",1


## Select team

In [11]:
decisions, captain_decisions, sub_decisions = select_team(prediction_df_sum.predicted_total_points_next_gameweek.values, prediction_df_sum.value.values, prediction_df_sum.position_id.values, prediction_df_sum.team.values, total_budget=100, sub_factor=0.15)

d:\studies\informatyka\semestr_3\fpl_ai_manager\venv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Total expected score = 269.25628447532654


In [12]:
prediction_df_sum

,name,element,position,value,team,GW,season,predicted_total_points_next_gameweek,next_4_GWs_matches,opponent_next_gameweek,position_id
0,Aaron Connolly,72,FWD,5.3,Brighton,29,2021-22,0.385969,5,"[Norwich, Arsenal, Tottenham, Manchester City,...",4
1,Aaron Cresswell,411,DEF,5.4,West Ham,30,2021-22,13.169465,4,"[Everton, Brentford, Burnley, Chelsea]",2
2,Aaron Lennon,575,MID,4.9,Burnley,29,2021-22,14.788957,6,"[Manchester City, Everton, Norwich, West Ham, ...",3
3,Aaron Ramsdale,559,GK,5.1,Arsenal,30,2021-22,10.169611,5,"[Crystal Palace, Brighton, Southampton, Chelse...",1
4,Aaron Ramsey,496,MID,4.4,Aston Villa,30,2021-22,-0.048319,3,"[Wolverhampton Wanderers, Tottenham, Leicester]",3
...,...,...,...,...,...,...,...,...,...,...,...
708,Zach Awe,705,DEF,4.0,Arsenal,30,2021-22,-0.181945,5,"[Crystal Palace, Brighton, Southampton, Chelse...",2
709,Zack Steffen,260,GK,4.4,Manchester City,29,2021-22,0.340187,4,"[Burnley, Liverpool, Brighton, Watford]",1
710,Zak Swanson,711,DEF,4.0,Arsenal,30,2021-22,-0.187515,5,"[Crystal Palace, Brighton, Southampton, Chelse...",2
711,Álvaro Fernández,556,GK,4.5,Brentford,30,2021-22,0.305082,4,"[Chelsea, West Ham, Watford, Tottenham]",1


In [13]:
print_selected_team(prediction_df_sum, decisions, captain_decisions, sub_decisions)

**Alexandre Lacazette** Predicted points = 21.78, Price = 8.4, Position = FWD

**Andrew Robertson** Predicted points = 19.09, Price = 7.2, Position = DEF

**Bukayo Saka** Predicted points = 24.37, Price = 6.7, Position = MID

**Fabian Schär** Predicted points = 16.04, Price = 4.3, Position = DEF

**Harry Kane** Predicted points = 24.99, Price = 12.4, Position = FWD

...

**Cédric Soares** Predicted points = 14.89, Price = 4.2, Position = DEF

**Daniel Amartey** Predicted points = 13.92, Price = 3.9, Position = DEF

**Martin Dubravka** Predicted points = 16.57, Price = 4.4, Position = GK


Captain:


**Kevin De Bruyne** Predicted points = 25.04, Price = 11.8, Position = MID